Usando o módulo DEAP
====================



## Introdução e importações



O módulo `DEAP` nos ajuda a construir algoritmos genéticos, fornecendo funções prontas para realizar diversas tarefas que nós estudamos durante o curso e muitas outras mais. Este módulo tem 3 submódulos que iremos utilizar:

-   `base`: este é o submódulo de base. É usando ele que iremos criar nossa caixa de ferramentas (`toolbox`) e nossa classe para computar a *fitness* dos indivíduos.

-   `tools`: este é o submódulo onde encontramos os operadores genéticos. Com as funções deste submódulo podemos criar indivíduos, populações, selecionar indivíduos, aplicar mutações e cruzamento.

-   `creator`: este é o submódulo que utilizaremos para criar coisas.

Para importar estes 3 submódulos basta rodar o código abaixo.



In [1]:
from deap import base
from deap import creator
from deap import tools

Vamos importar também o &ldquo;resolvedor&rdquo; de algoritmos genéticos do `DEAP`. Ficará mais claro isso ao longo deste notebook.



In [2]:
from deap.algorithms import eaSimple # resolve algoritmos já prontos

Agora vamos importar as funções e módulos que usaremos ao longo do notebook.



In [3]:
import numpy as np
from funcoes import gene_cb

## Problema das caixas binárias usando `DEAP` e a função `eaSimple`



Antes de iniciar o problema, vamos importar as funções necessárias e definir as constantes.



In [4]:
# relacionadas ao problema a ser resolvido
NUM_CAIXAS = 4

# relacionadas à busca
TAMANHO_POP = 6
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
CHANCE_MUTACAO_DE_CADA_GENE = 1 / NUM_CAIXAS # novidade! mutação pode ser a cada gene agora
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Vamos resolver o problema das caixas binárias usando o `DEAP`. Antes de mais nada, precisamos definir que tipo de problema nós temos em mãos, se é de maximização ou minimização. Nós sabemos que este é um problema de <u>maximização</u>. Para passar essa informação para o `DEAP` nós precisamos usar o `creator` e criar uma classe que tem como base o `base.Fitness`. Vamos dar o nome de `Fitness_max` para essa classe para nos lembrarmos que é a fitness de um problema de maximização. Se atente ao fato de que os pesos (`weights`) devem ser `(1.0,)` para problemas de maximização ou `(-1.0,)` para problemas de minimização (e devem ser escritos exatamente desta forma, não pode mudar aqui).



In [5]:
# Aqui, o nome `Fitness_max` fomos nós que demos, pode dar o nome que quiser
# O restante deve se manter constante caso esteja em um problema de maximização
creator.create("Fitness_max", base.Fitness, weights=(1.0,)) # objeto que cria fitness

# aqui que você define objetivos
# você pode ter mais de um objetivo e eles podem ter pesos diferentes
# é AQUI que você define se é umproblema de maximização ou minimização

Agora, vamos definir a estrutura de dados do nosso indivíduo. Para isso usamos o código abaixo. Observe que o nome `Individuo` fomos nós que demos. O segundo argumento que usamos abaixo foi `list`, para indicar que nosso indivíduo será uma lista. Finalmente, o último argumento (chamado `fitness`) deve ser o `Fitness_max` que criamos na célula acima. Se tivéssemos dado outro nome para o `Fitness_max` acima, deveríamos usar o mesmo nome aqui.



In [6]:
creator.create("Individuo", list, fitness=creator.Fitness_max)

# não criamos individuos, definimos individuos
# o primeiro item é o nome, o segundo é a ferramenta que você usar para criar o que quer
# o peso do indivíduo é o fitness
# indivíduo é uma lista

O próximo passo é criar a nossa caixa de ferramentas. A caixa de ferramenta, como o nome sugere, vai armazenar as ferramentas que usaremos para construir indivíduos, populações e operar geneticamente sobre eles. Para criar sua caixa de ferramentas faça o seguinte:



In [7]:
toolbox = base.Toolbox()

Afinal, o que é uma caixa de ferramentas sem ferramentas? Vamos adicionar ferramentas na nossa caixa! A estrutura para adicional uma ferramenta é sempre a mesma:

1.  Usamos o método `register` da nossa caixa de ferramentas.
2.  O primeiro argumento do método é o nome da ferramenta. Nós que escolhemos!
3.  O segundo argumento do método é a função que será executada quando usarmos a nossa ferramenta.
4.  Os demais argumentos do `register` são os argumentos que serão passados para a função do passo 3.

Vamos ver um exemplo criando uma ferramenta que cria um indivíduo:



In [8]:
toolbox.register(
    "individuo", tools.initRepeat, creator.Individuo, gene_cb, NUM_CAIXAS
)

# initRepeat define uma coisa n vezes
# no seu indivíduo você precisa de x genes e o valor deles
# número de vezes de armazenamento corresponde ao número de caixas

Indo pela ordem que comentamos acima, sabemos que o nome da nossa nova ferramenta é `individuo`. Quando essa ferramenta for usada, ela irá chamar a função `tools.initRepeat` com os seguintes argumentos: `creator.Individuo`, `gene_cb` e `NUM_CAIXAS`.

A função `tools.initRepeat` serve para criar um objeto através da repetição do uso de uma função. Neste exemplo, estamos criando um objeto do tipo `creator.Individuo` (que definimos lá em cima como sendo uma lista com uma certa propriedade de fitness, lembra?) e para criar este objeto nós rodamos a função `gene_cb` um total de `NUM_CAIXAS` de vezes e armazenamos o resultado na lista. Veja que essa estratégia é a mesma que usamos na função `individuo_cb` que criamos no `funcoes.py`.

Se mesmo com o parágrafo anterior ainda não ficou claro, veja a função `tools.initRepeat` sendo usada separadamente para ver como ela funciona.



In [9]:
uma_caixa_binaria = tools.initRepeat(list, gene_cb, NUM_CAIXAS)
print(uma_caixa_binaria)

[0, 0, 0, 1]


Vamos testar nossa ferramenta recém criada?



In [10]:
um_individuo_qualquer = toolbox.individuo()
print(um_individuo_qualquer)

[0, 0, 0, 1]


Agora veremos como criar uma ferramenta para criar populações:



In [11]:
toolbox.register(
    "populacao", tools.initRepeat, list, toolbox.individuo, TAMANHO_POP
)

# nossa população é composta por indivíduos
# então, vamos pedir para ele registrar indivíduos
# o número de individuos na população corresponde à constante 'tamanho_pop'
# ou seja, o número de registros será o número definido na constante

Veja que usamos novamente o `tools.initRepeat` aqui, onde iremos armazenar em uma lista os indivíduos gerados pela ferramenta `toolbox.individuo` que acabamos de criar logo acima. Vamos testar nossa nova ferramenta!



In [12]:
populacao = toolbox.populacao()
print(populacao)

[[1, 0, 0, 1], [0, 1, 1, 1], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0]]


Nossa próxima tarefa é criar a ferramenta que computa o fitness. Para isso precisamos escrever uma função que compute o fitness de um indivíduo. Já fizemos isso! É a função `funcao_objetivo_cb` do nosso arquivo `funcoes.py`. No entanto, o `DEAP` tem uma característica muito peculiar que ele <u>requer</u> que a função objetivo retorne uma tupla, e não um número. Essa peculiaridade é do próprio `DEAP`, não temos muito o que fazer a não ser seguir o que ele manda aqui. Para reescrever a função que já tínhamos escrito para retornar uma tupla é muito fácil! Veja abaixo.



In [13]:
def funcao_objetivo_cb(individuo):
    """Computa a função objetivo no problema das caixas binárias.

    Args:
      individiuo: lista contendo os genes das caixas binárias

    Return:
      Uma tupla com o valor representando a soma dos genes do individuo.
    """
    return (sum(individuo), )

# vírgula é sintaxe
# vai retornar um tupla

Vamos criar a ferramenta que computa o fitness. Aqui daremos o nome de `evaluate` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [14]:
toolbox.register("evaluate", funcao_objetivo_cb)

# esse nome não foi escolhido por nós
# esse nome NÃO pode mudar
# o evaluate calcula fitness
# NÃO funciona sem evaluate

Faltam apenas os operadores de seleção, cruzamento e mutação! Para seleção, vamos usar a seleção por torneio! O `DEAP` já tem essa função implementada em `tools.selTournament`. Aqui daremos o nome de `select` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [15]:
toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)

# já tem o torneio
# NÃO pode mudar o nome 'select'

Para cruzamento vamos utilizar o cruzamento de ponto simples. Novamente, o `DEAP` já tem essa função em `tools.cxOnePoint`. Aqui daremos o nome de `mate` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [16]:
toolbox.register("mate", tools.cxOnePoint)
#cross over
# não pode mudar mate também

Finalmente, para mutação vamos usar a função de flip bit. Essa função é mais &ldquo;inteligente&rdquo; do que a função de mutação que nós inventamos durante a disciplina. Se um gene for mutado e seu valor original era 0, ele passa a ter o valor de 1. Se um gene for mutado e seu valor original era 1, ele passa a ter o valor de 0. Isso melhora a convergência do problema quando comparado com a função de mutação que inventamos. Outro detalhe é que agora cada gene tem uma chance de mutar, ou seja, podemos ter zero, uma ou mais mutações por indivíduo! A chance de cada gene mutar é definida pelo parâmetro `indpb` abaixo. Aqui daremos o nome de `mutate` para essa ferramenta pois é o nome que o `DEAP` espera, por isso não traduzimos.



In [17]:
toolbox.register("mutate", tools.mutFlipBit, indpb=CHANCE_MUTACAO_DE_CADA_GENE)

O `DEAP` nos permite criar facilmente um hall da fama do tamanho que quisermos.



In [18]:
hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

E, para encerrar, temos também como acompanhar as estatísticas da nossa busca em tempo real! Para isso usamos o `tools.Statistics`. Veja um exemplo abaixo.



In [19]:
# pedíamos estatísticas
# me lembrou do que foi necessário na análise de performance

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

Pronto! Temos tudo que precisamos, podemos rodar nosso algoritmo genético usando o `eaSimple`.



In [20]:
populacao_inicial = toolbox.populacao()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_CRUZAMENTO,
    mutpb=CHANCE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

# verbose revela que você quer prints
# eaSimple vai encontrar a sua população usando o que você quer e coloca dentro do parênteses

gen	nevals	avg	std    	min	max
0  	6     	2  	0.57735	1  	3  
1  	6     	2.5	0.5    	2  	3  
2  	4     	2.83333	0.372678	2  	3  
3  	1     	3.16667	0.372678	3  	4  
4  	6     	3.16667	0.372678	3  	4  
5  	4     	3.33333	0.471405	3  	4  
6  	2     	3.83333	0.372678	3  	4  
7  	6     	4      	0       	4  	4  
8  	4     	4      	0       	4  	4  
9  	4     	4      	0       	4  	4  
10 	4     	4      	0       	4  	4  
11 	0     	4      	0       	4  	4  
12 	3     	3.33333	1.49071 	0  	4  
13 	4     	4      	0       	4  	4  
14 	2     	4      	0       	4  	4  
15 	2     	4      	0       	4  	4  
16 	2     	4      	0       	4  	4  
17 	2     	4      	0       	4  	4  
18 	0     	4      	0       	4  	4  
19 	2     	4      	0       	4  	4  
20 	4     	3.66667	0.745356	2  	4  
21 	4     	4      	0       	4  	4  
22 	2     	4      	0       	4  	4  
23 	3     	3.66667	0.745356	2  	4  
24 	2     	4      	0       	4  	4  
25 	0     	4      	0       	4  	4  
26 	2     	4      	0       	4  	4  
27 	2  

Podemos acessar o registro da nossa busca pela variável `log`.



In [21]:
print(log)

gen	nevals	avg    	std     	min	max
0  	6     	2      	0.57735 	1  	3  
1  	6     	2.5    	0.5     	2  	3  
2  	4     	2.83333	0.372678	2  	3  
3  	1     	3.16667	0.372678	3  	4  
4  	6     	3.16667	0.372678	3  	4  
5  	4     	3.33333	0.471405	3  	4  
6  	2     	3.83333	0.372678	3  	4  
7  	6     	4      	0       	4  	4  
8  	4     	4      	0       	4  	4  
9  	4     	4      	0       	4  	4  
10 	4     	4      	0       	4  	4  
11 	0     	4      	0       	4  	4  
12 	3     	3.33333	1.49071 	0  	4  
13 	4     	4      	0       	4  	4  
14 	2     	4      	0       	4  	4  
15 	2     	4      	0       	4  	4  
16 	2     	4      	0       	4  	4  
17 	2     	4      	0       	4  	4  
18 	0     	4      	0       	4  	4  
19 	2     	4      	0       	4  	4  
20 	4     	3.66667	0.745356	2  	4  
21 	4     	4      	0       	4  	4  
22 	2     	4      	0       	4  	4  
23 	3     	3.66667	0.745356	2  	4  
24 	2     	4      	0       	4  	4  
25 	0     	4      	0       	4  	4  
26 	2     	4      	0       	

Podemos acessar o nosso hall da fama da seguinte maneira:



In [22]:
print(hall_da_fama.items)

[[1, 1, 1, 1]]


<hr>

## Anotações
Esse mano já tem mutações embutidas;
<br> Conseguimos trabalhar com a chance individual de um gene ser mutado;
<br> Deap adora tuplas;
<br> Se fosse um problema de minimização, você coloca -1 no peso da fitness;
<br>Após rodar o notebook, ele rodou de novo (não reiniciar, mas indo célula por célula);
<br> O deap não foi feito para resolver mais de um problema mais de uma vez, a dica é sempre REINICIAR;
<br>Não controlamos o critério de parada com o eaSimple.